#Encontrar un DataFrame con las ubicaciones de MiBici en el estado de Jalisco

El servicio de MiBici es ofrecido en las ciudades de Guadalajara (GDL), Zapopán (ZPN) y Tlaquepaque(TLQ).
Buscamos un DataFrame que nos de la inforación para ubicar las estaciones de bicicletas para renta. Para esto se extrae información de la API del proyecto global Citybik.

In [1]:
## Imports necesarios
import requests
import pandas as pd
import numpy as np

In [2]:
endpoint = 'https://api.citybik.es/v2/networks/mibici-guadalajara'
r = requests.get(endpoint)
r.status_code

200

In [3]:
json = r.json()
Info_MIBICI =  pd.json_normalize(json)
Info_MIBICI

,network.company,network.gbfs_href,network.href,network.id,network.location.city,network.location.country,network.location.latitude,network.location.longitude,network.name,network.stations
0,"[BKT bici publica S.A. de C.V., PBSC Urban Sol...",https://guad.publicbikesystem.net/ube/gbfs/v1/,/v2/networks/mibici-guadalajara,mibici-guadalajara,Guadalajara,MX,20.673788,-103.370433,MIBICI,"[{'empty_slots': 11, 'extra': {'address': '(ZP..."


A primera vista se nos ofrecen datos de identificación en general del servicio MiBici, esto para separlo de otros servicios en el mundo, la informacion que nos interesa sobre las estaciones se encuentra en el campo "network.stations"

In [4]:
Info_MIBICI["network.stations"][0][0] #observamos los datos que se incluyen por estación

{'empty_slots': 11,
 'extra': {'address': '(ZPN-043) C. Ermita / C. Del Árbol',
  'last_updated': 1663542024,
  'payment': ['key', 'transitcard', 'creditcard', 'phone'],
  'payment-terminal': True,
  'renting': 1,
  'returning': 1,
  'uid': '137'},
 'free_bikes': 0,
 'id': '1f0f3254623c37ed18c34f6a7dca449c',
 'latitude': 20.65938,
 'longitude': -103.40195,
 'name': '(ZPN-043) C. Ermita / C. Del Árbol',
 'timestamp': '2022-09-18T23:00:47.649000Z'}

La mayoría son datos sobre el uso operativo y comercial de las estaciones, sólo nos interesan los registros de 'latitude', 'longitud' y 'name'. Éste último tiene la misma información que 'address' dentro de 'extra', por lo que además de ser redundante, a 'name' lo relacionamos por lo pronto con la dirección de la estación.

Definimos 3 listas para los campos de nuestro interés. Con un Ciclo for extraemos la información correspondiente de cada estación.

In [5]:
#definimos listas
direcciones = []
latitudes = []
longitudes = []

for i in range(0,299): # va a tomar las 299 estaciones registradas 
  direcciones.append(Info_MIBICI["network.stations"][0][i]['name'])
  latitudes.append(Info_MIBICI["network.stations"][0][i]['latitude'])
  longitudes.append(Info_MIBICI["network.stations"][0][i]['longitude'])

Cada lista de puede convertir a una serie de Pandas de la siguiente forma:

In [6]:
direcciones_serie = pd.Series(direcciones) 
latitudes_serie = pd.Series(latitudes)
longitudes_serie = pd.Series(longitudes)

Buscamos crear un DataFrame con estas series, para esto primero hay que definir un diccionario.

In [7]:
diccionario_ubicaciones = {'Dirección': direcciones_serie, "Latitud": latitudes_serie, "Longitud": longitudes_serie}

Obtenemos nuestro DataFrame con los datos de ubicación para cada estación de bicicletas. 

In [8]:
df_ubicaciones = pd.DataFrame(diccionario_ubicaciones)

#Exploramos el dataframe df_ubicaciones

In [9]:
df_ubicaciones

,Dirección,Latitud,Longitud
0,(ZPN-043) C. Ermita / C. Del Árbol,20.659380,-103.401950
1,(ZPN-045) Av. Tizoc / C. Iztaccíhuatl,20.657010,-103.404640
2,(ZPN-047) Av. Tizoc / Av. López Mateos,20.653810,-103.401340
3,(ZPN-067) C. Chimalhuacán / Av. López Mateos,20.651592,-103.402769
4,(GDL-218) C.Hércules / Av. De la Arboleda,20.664447,-103.385311
...,...,...,...
294,(GDL-033) Av. Hidalgo / C. Pedro Loza,20.677419,-103.347983
295,(GDL-032) C. Degollado / Av. Hidalgo,20.677263,-103.344908
296,(GDL-134) C. Agustín Yáñez / C. Fco Ugarte,20.670160,-103.384480
297,(ZPN-051) Av. Periférico G. Morín / Av. J. Pa,20.738370,-103.380570


In [10]:
df_ubicaciones.head()

,Dirección,Latitud,Longitud
0,(ZPN-043) C. Ermita / C. Del Árbol,20.659380,-103.401950
1,(ZPN-045) Av. Tizoc / C. Iztaccíhuatl,20.657010,-103.404640
2,(ZPN-047) Av. Tizoc / Av. López Mateos,20.653810,-103.401340
3,(ZPN-067) C. Chimalhuacán / Av. López Mateos,20.651592,-103.402769
4,(GDL-218) C.Hércules / Av. De la Arboleda,20.664447,-103.385311


In [11]:
df_ubicaciones.tail()

,Dirección,Latitud,Longitud
294,(GDL-033) Av. Hidalgo / C. Pedro Loza,20.677419,-103.347983
295,(GDL-032) C. Degollado / Av. Hidalgo,20.677263,-103.344908
296,(GDL-134) C. Agustín Yáñez / C. Fco Ugarte,20.670160,-103.384480
297,(ZPN-051) Av. Periférico G. Morín / Av. J. Pa,20.738370,-103.380570
298,(ZPN-057) C. Escultores / Av. Guadalupe,20.663707,-103.417817


In [12]:
df_ubicaciones.shape

(299, 3)

In [13]:
df_ubicaciones.columns

Index(['Dirección', 'Latitud', 'Longitud'], dtype='object')

In [14]:
df_ubicaciones.dtypes

Dirección     object
Latitud      float64
Longitud     float64
dtype: object

In [15]:
df_ubicaciones.isna().sum()

Dirección    0
Latitud      0
Longitud     0
dtype: int64

In [16]:
df_ubicaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Dirección  299 non-null    object 
 1   Latitud    299 non-null    float64
 2   Longitud   299 non-null    float64
dtypes: float64(2), object(1)
memory usage: 7.1+ KB


Nuestro DataFrame se encuentra bien definido en el sentido de que no hay NaNs, ni valores nulos, y los campos tienen el tipo correcto de dato, es decir, "Dirección" es 'object' mientras que tanto "Latitud" como "Longitud" son 'float64', el equivalente de pandas a float.

Sin embargo, identificamos que los que se definió como dirección en la API, tiene primero una zona y luego dos calles, por lo que podemos realizar esta separación de columnas. 

In [17]:
df_ubicaciones[['Zona','Calles']] = df_ubicaciones['Dirección'].str.split('\s',1, expand = True) #Va a separar sólo en el primer espacio (\s)


In [18]:
df_ubicaciones.head() #Visializamos el DataFrame con las nuevas columnas agregadas

,Dirección,Latitud,Longitud,Zona,Calles
0,(ZPN-043) C. Ermita / C. Del Árbol,20.659380,-103.401950,(ZPN-043),C. Ermita / C. Del Árbol
1,(ZPN-045) Av. Tizoc / C. Iztaccíhuatl,20.657010,-103.404640,(ZPN-045),Av. Tizoc / C. Iztaccíhuatl
2,(ZPN-047) Av. Tizoc / Av. López Mateos,20.653810,-103.401340,(ZPN-047),Av. Tizoc / Av. López Mateos
3,(ZPN-067) C. Chimalhuacán / Av. López Mateos,20.651592,-103.402769,(ZPN-067),C. Chimalhuacán / Av. López Mateos
4,(GDL-218) C.Hércules / Av. De la Arboleda,20.664447,-103.385311,(GDL-218),C.Hércules / Av. De la Arboleda


Ahora nos deshacemos de la columna original "Dirección"

In [19]:
df_ubicaciones = df_ubicaciones.drop(columns='Dirección')

In [20]:
df_ubicaciones.columns #comprobamos las columnas que quedaron en el DataFrame

Index(['Latitud', 'Longitud', 'Zona', 'Calles'], dtype='object')

In [21]:
df_ubicaciones = df_ubicaciones.reindex(columns=['Calles','Zona','Latitud','Longitud']) #Reorganizamos el orden de las columnas para que el texto quede primero

Exploramos el DataFrame ya reorganizado con las nuevas columnas

In [22]:
df_ubicaciones.columns

Index(['Calles', 'Zona', 'Latitud', 'Longitud'], dtype='object')

In [23]:
df_ubicaciones.dtypes

Calles       object
Zona         object
Latitud     float64
Longitud    float64
dtype: object

In [24]:
df_ubicaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Calles    299 non-null    object 
 1   Zona      299 non-null    object 
 2   Latitud   299 non-null    float64
 3   Longitud  299 non-null    float64
dtypes: float64(2), object(2)
memory usage: 9.5+ KB


No se han encontrado irregularidades en los tipos de datos ni tamaño de las filas o columnas. 

Ahora vamos a explorar cuántas estaciones hay por cada municipio registrado. (Guadalajara (GDL), Zapopán (ZPN), Tlaquepaque(TLQ)) mediante str.contains de Pandas, que devuelve 'True' si encuentra la coincidencia de caracteres.

In [25]:
estaciones_ZPN = df_ubicaciones["Zona"].str.contains('ZPN').sum()
estaciones_ZPN

61

In [26]:
estaciones_TLQ = df_ubicaciones["Zona"].str.contains('TLQ').sum()
estaciones_TLQ

12

In [27]:
estaciones_GDL = df_ubicaciones["Zona"].str.contains('GDL').sum()
estaciones_GDL

226

In [28]:
total_estaciones = np.sum([estaciones_ZPN,estaciones_TLQ,estaciones_GDL])
porcentaje_estaciones_GDL = np.multiply(np.divide(estaciones_GDL,total_estaciones),100)
porcentaje_estaciones_GDL


75.58528428093646

Se observa mayor presencia de estaciones de bicicletas en Guadalajara (75.58 %).

Con esto ya tenemos un dataframe disponible para que cuando se proponga una ubicación para la tienda de bicicletas se puede analizar si es buena opción, en base a la cercanía con las estaciones del servicio MiBici. La presencia de estaciones podría ser poco estimulante para la venta de bicicletas pero quizás sí para accesorios. 

In [29]:
df_ubicaciones

,Calles,Zona,Latitud,Longitud
0,C. Ermita / C. Del Árbol,(ZPN-043),20.659380,-103.401950
1,Av. Tizoc / C. Iztaccíhuatl,(ZPN-045),20.657010,-103.404640
2,Av. Tizoc / Av. López Mateos,(ZPN-047),20.653810,-103.401340
3,C. Chimalhuacán / Av. López Mateos,(ZPN-067),20.651592,-103.402769
4,C.Hércules / Av. De la Arboleda,(GDL-218),20.664447,-103.385311
...,...,...,...,...
294,Av. Hidalgo / C. Pedro Loza,(GDL-033),20.677419,-103.347983
295,C. Degollado / Av. Hidalgo,(GDL-032),20.677263,-103.344908
296,C. Agustín Yáñez / C. Fco Ugarte,(GDL-134),20.670160,-103.384480
297,Av. Periférico G. Morín / Av. J. Pa,(ZPN-051),20.738370,-103.380570
